In [1]:
import pandas as pd
import random
import json


def data_random_extraction(path, n, n_data):
    column_names = ["article_text", "abstract_text", "section_names", "sections"]
    indices = []
    
    for i in range(n):
        indices.append(i)
    
    dataframe = pd.DataFrame(columns=column_names, index=indices)
    
    file = open(path, 'r')
    Lines = file.readlines()
    index = set()
    data_index = 0
    c = 0
    
    index = set(random.sample(range(n_data), n))
    
    for line in Lines:
        if c in index:
            cur = json.loads(line)
            dataframe.loc[data_index]['article_text'] = cur['article_text']
            dataframe.loc[data_index]['abstract_text'] = cur['abstract_text']
            dataframe.loc[data_index]['section_names'] = cur['section_names']
            dataframe.loc[data_index]['sections'] = cur['sections']
            data_index += 1
            
        c += 1
        
    return dataframe

In [2]:
def data_extraction(path, n, n_data):
    column_names = ["article_text", "abstract_text", "section_names"]
    indices = []
    
    for i in range(n):
        indices.append(i)
    
    dataframe = pd.DataFrame(columns=column_names, index=indices)
    
    file = open(path, 'r')
    Lines = file.readlines()
    # data_index = 0
    row = 0
    
    for line in Lines:
        if row < n:
            cur = json.loads(line)
            
            corpus = ''
            for sentence in cur['abstract_text']:
                corpus += sentence
            dataframe.loc[row]['abstract_text'] = corpus.replace('<S> ', '').replace(' </S>', '')
            
            corpus = ''
            for section in cur['sections']:
                for sentence in section:
                    corpus += sentence
            dataframe.loc[row]['article_text'] = corpus
            
            dataframe.loc[row]['section_names'] = cur['section_names']
            
            row += 1
        
        else: break
        
    return dataframe

In [5]:
n_train = 203037
n_test = 6440
n_val = 6436

new_train = data_extraction('C:\\Users\\ivanj\\Desktop\\Grad_School\\ESE527\\arxiv-dataset\\train.txt', n_train//13, n_train)
new_test = data_extraction('C:\\Users\\ivanj\\Desktop\\Grad_School\\ESE527\\arxiv-dataset\\test.txt', n_test//13, n_test)
new_val = data_extraction('C:\\Users\\ivanj\\Desktop\\Grad_School\\ESE527\\arxiv-dataset\\val.txt', n_val//13, n_val)

In [12]:
n_train + n_test + n_val

215913

In [11]:
new_train

,article_text,abstract_text,section_names
0,additive models @xcite provide an important fa...,additive models play an important role in semi...,"[introduction, main results on learning rates,..."
1,the leptonic decays of a charged pseudoscalar ...,"we have studied the leptonic decay @xmath0 , v...","[[sec:introduction]introduction, [sec:detector..."
2,the transport properties of nonlinear non - eq...,"in 84 , 258 ( 2000 ) , mateos conjectured that...","[introduction, regularity and chaos in single-..."
3,studies of laser beams propagating through tur...,the effect of a random phase diffuser on fluct...,"[introduction, the method of photon distributi..."
4,the so - called `` nucleon spin crisis '' rais...,with a special intention of clarifying the und...,"[introduction, model lagrangian with pion mass..."
...,...,...,...
15613,the scattering of an unpolarized electron beam...,the spin polarized charge transport is systema...,"[introduction, model, stationary current, spin..."
15614,the variability of the light - curve of a lens...,the influence of microlensing in the profiles ...,"[introduction, microlensing effects at high op..."
15615,one of the main contemporary challenges of the...,the dynamics of two competing species in a fin...,"[introduction, intuitive arguments, glossary a..."
15616,one of the most impressive successes of the st...,we determine the hadronic light - by - light s...,"[introduction[sec:intro], conclusions[sec:conc..."


In [9]:
dataset_path = 'C:\\Users\\ivanj\\Desktop\\Grad_School\\ESE527\\arxiv-dataset'

In [10]:
new_train.to_csv(dataset_path+'\\new_train.csv')
new_test.to_csv(dataset_path+'\\new_test.csv')
new_val.to_csv(dataset_path+'\\new_val.csv')

In [ ]:
# The following code were adepted from 
# https://medium.com/@ferlatti.aldo/fine-tuning-a-chat-summarizer-c18625bc817d

import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_metric, load_from_disk
import numpy as np
import nltk
# nltk.download('punkt')

In [ ]:
def preprocess_data(data_to_process):
    #get all the dialogues
    inputs = [dialogue for dialogue in data_to_process['article_text']]
    #tokenize the dialogues
    model_inputs = tokenizer(inputs, max_length=max_input, padding='max_length', truncation=True)
    #tokenize the summaries
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(data_to_process['abstract_text'], max_length=max_target, padding='max_length', truncation=True)

    #set labels
    model_inputs['labels'] = targets['input_ids']
    #return the tokenized data
    #input_ids, attention_mask and labels
    return model_inputs

In [77]:
data = load_dataset("/Users/yifeng/Desktop/Grad/ESE527/arxiv-dataset/", data_files={"train": 'new_train.csv', 'test': 'new_test.csv', "validate": "new_val.csv"})
tokenize_data = data.map(preprocess_data, batched = True)

Found cached dataset csv (/Users/yifeng/.cache/huggingface/datasets/csv/arxiv-dataset-935f63ede0f5d4ff/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

In [85]:
max_input = 512
max_target = 128
batch_size = 3
model_checkpoints = "facebook/bart-large-xsum"
metric = load_metric('rouge')

tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

/var/folders/9b/gzjjnwn16kb_zpz40lt9bhx40000gn/T/ipykernel_80447/2736189134.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')


In [87]:
def compute_rouge(pred):
    predictions, labels = pred
    #decode the predictions
    decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    #decode labels
    decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    #compute results
    res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
    #get %
    res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

    pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    res['gen_len'] = np.mean(pred_lens)

    return {k: round(v, 4) for k, v in res.items()}

In [88]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

args = Seq2SeqTrainingArguments(
    'paper-summary', #save directory
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size= 2,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=3,
    fp16=False #available only with CUDA
    )

trainer = Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validate'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

trainer.train()

loading configuration file config.json from cache at /Users/yifeng/.cache/huggingface/hub/models--facebook--bart-large-xsum/snapshots/2179ab81d3f133e639f2957aec5380e9d56b2783/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-large-xsum",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "eos_token_ids": [
    2
  ],
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,No log,3.838084,20.341100,5.428500,13.835300,13.796700,36.100000
1,No log,3.712173,23.315500,6.940200,14.419200,14.399700,45.200000
2,No log,3.675076,28.463500,7.729100,17.125200,17.163400,60.900000


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: abstract_text, article_text, section_names, Unnamed: 0. If abstract_text, article_text, section_names, Unnamed: 0 are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 2
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1

TrainOutput(global_step=6, training_loss=4.837347666422526, metrics={'train_runtime': 1348.5288, 'train_samples_per_second': 0.022, 'train_steps_per_second': 0.004, 'total_flos': 30339464429568.0, 'train_loss': 4.837347666422526, 'epoch': 2.8})

In [89]:
test = new_test.loc[0]['article_text']
model_inputs = tokenizer(test,  max_length=max_input, padding='max_length', truncation=True)
raw_pred, _, _ = trainer.predict([model_inputs])
tokenizer.decode(raw_pred[0])

***** Running Prediction *****
  Num examples = 1
  Batch size = 2
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



'</s><s>for about 20 years the problem of properties of short - term changes of solar activity has been considered.many investigators studied the short - time periodicities of the various indices of the solar activity.several periodicities were detected, but the periodicities about 155 days and from the interval of @x</s>'

In [90]:
tokenizer.decode(raw_pred[0])

'</s><s>for about 20 years the problem of properties of short - term changes of solar activity has been considered.many investigators studied the short - time periodicities of the various indices of the solar activity.several periodicities were detected, but the periodicities about 155 days and from the interval of @x</s>'